In [1]:
import datetime
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yfin

from numpy import linalg as LA

In [2]:
# Download stock prices from Yahoo Finance and set the time period for download
start = datetime.date(2018, 1, 2)
end = datetime.date(2023, 12, 31)

# Get data
# To switch between live data download and local data loading, 
# uncomment the desired line and comment out the other.
# Get Amazon, Ford and Bitcoin data
#stocks = yfin.download(["AAPL", "F", "WMT","PFE"], start, end, auto_adjust = False)["Adj Close"]
stocks = pd.read_csv('C:\\Users\\DELL\\OneDrive\\Desktop\\M.Sc.(WQU)\\C2_Financial_Data\\M3_Portfolios_and_tick_data\\L4\\FD_M3_L4_stocks.csv', index_col='Date', parse_dates=True)\
                    .loc[start:end].reindex(columns=["AAPL", "F", "WMT","PFE"])

stocks.head()


,AAPL,F,WMT,PFE
Date,,,,
2018-01-02,40.341885,8.263117,28.914423,24.229599
2018-01-03,40.334858,8.328384,29.166639,24.409126
2018-01-04,40.522213,8.471978,29.193027,24.462317
2018-01-05,40.983566,8.615572,29.366074,24.508865
2018-01-08,40.831348,8.582937,29.800114,24.236256


In [3]:
stocks.index = pd.to_datetime(stocks.index).strftime("%Y-%m-%d")

In [4]:
# In order to calculate the returns of the stocks, we need to drop the NA rows.
stocks_returns = stocks[["AAPL", "F", "WMT","PFE"]].dropna().pct_change()
stocks_returns = stocks_returns.dropna()
stocks_returns.head()

,AAPL,F,WMT,PFE
Date,,,,
2018-01-03,-0.000174,0.007899,0.008723,0.007409
2018-01-04,0.004645,0.017241,0.000905,0.002179
2018-01-05,0.011385,0.016949,0.005928,0.001903
2018-01-08,-0.003714,-0.003788,0.014780,-0.011123
2018-01-09,-0.000115,-0.005323,-0.012007,-0.001098


In [5]:
# Perform SVD for stock returns
U, s, VT = np.linalg.svd(stocks_returns)

In [6]:
# Present the result
print("Stock Returns Matrix Dimension:")
print(stocks_returns.shape)
print("\nDimension of Matrix U:")
print(U.shape)
print("\nSingular values:")
print(s)
print("\nDimension of Matrix V^T:")
print(VT.shape)

Stock Returns Matrix Dimension:
(1508, 4)

Dimension of Matrix U:
(1508, 1508)

Singular values:
[1.11696739 0.72489554 0.55820564 0.47159228]

Dimension of Matrix V^T:
(4, 4)


In [7]:
# Standardize stock returns dataset
stocks_returns_means = stocks_returns.mean()
stocks_returns_stds = stocks_returns.std()
standardized_returns = (stocks_returns - stocks_returns_means) / stocks_returns_stds
standardized_returns.head()

,AAPL,F,WMT,PFE
Date,,,,
2018-01-03,-0.070358,0.286261,0.584133,0.446787
2018-01-04,0.171148,0.647641,0.029986,0.124438
2018-01-05,0.508925,0.636338,0.386009,0.107413
2018-01-08,-0.247757,-0.165773,1.013486,-0.695380
2018-01-09,-0.067374,-0.225159,-0.885164,-0.077534


In [8]:
# Calculate covariance for standardized return matrix
standardized_returns_dvd_sqrt_n=(standardized_returns/math.sqrt(len(standardized_returns)-1))
standardized_returns_cov = standardized_returns_dvd_sqrt_n.T@standardized_returns_dvd_sqrt_n
standardized_returns_cov

,AAPL,F,WMT,PFE
AAPL,1.000000,0.376460,0.365246,0.324438
F,0.376460,1.000000,0.185248,0.224384
WMT,0.365246,0.185248,1.000000,0.296375
PFE,0.324438,0.224384,0.296375,1.000000


In [9]:
# Use SVD to calculate eigenvectors and eigenvalues of the covariance matrix of standardized returns
U_st_return, s_st_return, VT_st_return = np.linalg.svd(standardized_returns_dvd_sqrt_n)
print("\nSquared Singular values (eigenvalues):")
print(s_st_return**2)
print("\nMatrix V (eigenvectors)")
print(VT_st_return.T)


Squared Singular values (eigenvalues):
[1.8947492  0.83555338 0.71064931 0.55904811]

Matrix V (eigenvectors)
[[ 0.56634769  0.14011624 -0.21637439 -0.78281534]
 [ 0.45964766  0.75759291  0.0183351   0.46307756]
 [ 0.48586769 -0.53226779 -0.55857187  0.41063494]
 [ 0.48156713 -0.35087237  0.80052696  0.06432933]]


In [10]:
# Use the method from Module 1 Lesson 4 to calculate eigenvectors and 
# eigenvalues of the covariance matrix of standardized returns
eigenvalues, eigenvectors = LA.eig(standardized_returns_cov)
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]
eigenvalues

array([1.8947492 , 0.83555338, 0.71064931, 0.55904811])

In [11]:
eigenvectors

array([[-0.56634769,  0.14011624, -0.21637439, -0.78281534],
       [-0.45964766,  0.75759291,  0.0183351 ,  0.46307756],
       [-0.48586769, -0.53226779, -0.55857187,  0.41063494],
       [-0.48156713, -0.35087237,  0.80052696,  0.06432933]])